In [94]:
import pandas as pd
import plotly.express as px
from numpy.random import randint
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
import math
import geopy.distance
from sklearn.linear_model import LinearRegression
from datetime import datetime, timedelta, timezone
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

# Importing the dataset

In [95]:
%%time
uber = pd.read_csv("uber.csv")

CPU times: user 162 ms, sys: 3.97 ms, total: 166 ms
Wall time: 165 ms


# Cleaning the data:
Some latitude and longitude coordinates are not in this world so they are removed

After removing the impossible data points, we can see that there are still some latitude and longitude coordinates that are far from the others, some in Antartica and some in the ocean, while the vast majority of the data points are near New York.

In order to remove these points we will remove points which have > 2 standard deviations from the mean in both longitude and latitude.

In [96]:
# fig = figpx.scatter_geo(uber, lon="pickup_longitude", lat="pickup_latitude")
# fig.show()

In [97]:
class CalculateDistanceTraveled(BaseEstimator, TransformerMixin):
    def calc_distance(self, x):
        coords_1 = (x.pickup_latitude, x.pickup_longitude)
        coords_2 = (x.dropoff_latitude, x.dropoff_longitude)
        try:
            miles = geopy.distance.distance(coords_1, coords_2).miles
        except:
            return -1
        return miles
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        X["distance_traveled"] = X.apply(lambda x : self.calc_distance(x), axis=1)
        return X

In [98]:
class RemoveOutliers(BaseEstimator, TransformerMixin):
    def __init__(self, column_name):
        self.column_name = column_name
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        mean = X[self.column_name].mean()
        std = X[self.column_name].std()
        X = X[(X[self.column_name] < mean + 3*std) & (X[self.column_name] > mean - 3*std)]
        return X

In [99]:
class CalculateDistanceTraveled(BaseEstimator, TransformerMixin):
    def calc_distance(self, x):
        coords_1 = (x.pickup_latitude, x.pickup_longitude)
        coords_2 = (x.dropoff_latitude, x.dropoff_longitude)
        try:
            miles = geopy.distance.great_circle(coords_1, coords_2).miles
        except:
            return -1
        return miles
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        X["distance_traveled"] = X.apply(lambda x : self.calc_distance(x), axis=1)
        return X

In [100]:
class TransformDateTime(BaseEstimator, TransformerMixin):
    def __init__(self, column_name):
        self.column_name = column_name
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        X[self.column_name] = pd.to_datetime(X[self.column_name])
        return X

In [101]:
class ExtractWeekday(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        X["weekday"] = X["pickup_datetime"].dt.weekday
        return X

In [102]:
pipeline = Pipeline([
    ("RemoveOutliers-pickup_longitude", RemoveOutliers("pickup_longitude")),
    ("RemoveOutliers-pickup_latitude", RemoveOutliers("pickup_latitude")),
    ("RemoveOutliers-dropoff_latitude", RemoveOutliers("dropoff_latitude")),
    ("RemoveOutliers-dropoff_longitude", RemoveOutliers("dropoff_longitude")),
    ("CalculateDistanceTraveled", CalculateDistanceTraveled()),
    ("CustomTransformer_CalculatingDistanceTraveled", CalculateDistanceTraveled()),
    ("RemovingOutliers-distance_traveled", RemoveOutliers("distance_traveled")),
    ("TransformDateTime-pickup_datetime", TransformDateTime("pickup_datetime")),
    ("ExtractWeekday", ExtractWeekday()),
])

In [103]:
%%time
uber = pipeline.fit_transform(uber)

CPU times: user 18.3 s, sys: 11.8 ms, total: 18.4 s
Wall time: 18.4 s


In [104]:
uber["pickup_datetime"] = uber["pickup_datetime"].dt.hour

In [105]:
uber.drop(["key", "Unnamed: 0"], axis=1, inplace=True)

In [106]:
uber.head()

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,distance_traveled,weekday
0,7.5,19,-73.999817,40.738354,-73.999512,40.723217,1,1.045970,3
1,7.7,20,-73.994355,40.728225,-73.994710,40.750325,1,1.527078,4
2,12.9,21,-74.005043,40.740770,-73.962565,40.772647,1,3.129464,0
3,5.3,8,-73.976124,40.790844,-73.965316,40.803349,3,1.032524,4
4,16.0,17,-73.925023,40.744085,-73.973082,40.761247,5,2.780920,3


# Modelling

In [107]:
target = uber.fare_amount
params = uber.drop(["fare_amount"], axis=1)

In [108]:
model = RandomForestRegressor(n_estimators=10, max_depth=10, random_state=42)
forest_scores = cross_val_score(model, params, target, scoring="neg_mean_squared_error", cv=10)

In [111]:
forest_scores.mean()*-1

17.829669162743393